In [16]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (20, 10)

## Introduction 

In [17]:
smartphones = pd.read_csv('../Data sets/smartphone_raw.csv')

In [18]:
smartphones.shape

(338018, 5)

In [19]:
smartphones.head()

,event_time,event_type,category_id,price,user_id
0,2019-10-01 00:02:14 UTC,purchase,2053013555631882655,130.76,543272936
1,2019-10-01 00:04:37 UTC,purchase,2053013555631882655,642.69,551377651
2,2019-10-01 00:10:08 UTC,purchase,2053013555631882655,515.67,524325294
3,2019-10-01 00:14:14 UTC,purchase,2053013555631882655,463.31,555083442
4,2019-10-01 02:19:10 UTC,purchase,2053013555631882655,736.18,515246296


In [20]:
smartphones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338018 entries, 0 to 338017
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   event_time   338018 non-null  object 
 1   event_type   338018 non-null  object 
 2   category_id  338018 non-null  int64  
 3   price        338018 non-null  float64
 4   user_id      338018 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 12.9+ MB


## Data cleaning

In [21]:
round((smartphones.isnull().sum() / smartphones.shape[0]) * 100, 2)

event_time     0.0
event_type     0.0
category_id    0.0
price          0.0
user_id        0.0
dtype: float64

Данные чистые. Заполнять искусственно не нужно.

## Data Preparation

Удалим время у фичи `event_time`, и переименуем ее в `event_date`

In [22]:
smartphones['event_time'] = pd.to_datetime(smartphones['event_time'])

In [23]:
smartphones['event_time'] = smartphones['event_time'].map(lambda x: 10000*x.year + 100*x.month + x.day)

In [24]:
smartphones = smartphones.rename(columns={'event_time': 'event_date'})

In [25]:
smartphones.head()

,event_date,event_type,category_id,price,user_id
0,20191001,purchase,2053013555631882655,130.76,543272936
1,20191001,purchase,2053013555631882655,642.69,551377651
2,20191001,purchase,2053013555631882655,515.67,524325294
3,20191001,purchase,2053013555631882655,463.31,555083442
4,20191001,purchase,2053013555631882655,736.18,515246296


Удалю колонки, которые нам уже ненужные колонки - `category_id` и `event_type`

In [26]:
smartphones = smartphones.drop(['category_id', 'event_type'], axis=1)

In [27]:
smartphones.head()

,event_date,price,user_id
0,20191001,130.76,543272936
1,20191001,642.69,551377651
2,20191001,515.67,524325294
3,20191001,463.31,555083442
4,20191001,736.18,515246296


In [28]:
smartphones.to_csv('../Data sets/smartphones_cleaned.csv', index=False)

И снова сократил с 23,7 MB до 12 MB. И притом оставили только нужные нам данные.

### RFM

Для кластеризации пользователей я буду использовать сегментацию **RFM**.

**RFM** (**R**ecency **F**requency **M**onetary) - разделение клиентов на сегменты от степени их лояльности.

У такой сегментации всего 3 колонки:

**R**ecency (давность) - давность прошлой сделки, сколько времени прошло с прошлой покупки. Предполагается, чем меньше эта метрика, тем больше вероятность будущей, повторной покупки.

**F**requency (частота) - кол-во покупок. Больше покупок, больше вероятность возвращение клиента.

**M**onetary (денежная масса, деньги) - сумма сделок. Чем больше потратил клиент, тем больше вероятность возвращение клиента.

### Data Transformation 

In [29]:
# Для начала найдем уникальных юзеров 

users = pd.DataFrame({'user_id': smartphones['user_id'].unique()})

In [30]:
users.head()

,user_id
0,543272936
1,551377651
2,524325294
3,555083442
4,515246296


In [31]:
# Их кстати 
users.shape[0]

160437

### Соберем дату последней покупке юзеров

In [89]:
last_ordered_date = smartphones.groupby(['user_id'])['event_date'].agg('max')

In [90]:
zipped_last_ordered_date = zip(last_ordered_date.index, last_ordered_date.values)

In [91]:
%%time
for index, value in zipped_last_ordered_date:
    users.loc[users['user_id'] == index, 'last_ordered_date'] = value

CPU times: user 2min 19s, sys: 699 ms, total: 2min 20s
Wall time: 2min 20s


In [92]:
users.head()

,user_id,n_orders,last_ordered_date
0,543272936,32.0,20191031.0
1,551377651,13.0,20191025.0
2,524325294,3.0,20191008.0
3,555083442,2.0,20191005.0
4,515246296,8.0,20191024.0


### Теперь соберем информацию о кол-во покупок.

In [93]:
n_orders = smartphones.groupby(['user_id']).agg('count')['price']

In [94]:
zipped_n_orders = zip(n_orders.index, n_orders.values)

In [95]:
%%time
for index, value in zipped_n_orders:
    users.loc[users['user_id'] == index, 'n_orders'] = value

CPU times: user 2min 26s, sys: 883 ms, total: 2min 26s
Wall time: 2min 27s


In [96]:
users.head()

,user_id,n_orders,last_ordered_date
0,543272936,32.0,20191031.0
1,551377651,13.0,20191025.0
2,524325294,3.0,20191008.0
3,555083442,2.0,20191005.0
4,515246296,8.0,20191024.0


### Просуммируем цены заказов.

In [97]:
amount = smartphones.groupby(['user_id'])['price'].agg('sum')

In [98]:
zipped_amount = zip(amount.index, amount.values)

In [99]:
%%time
for index, value in zipped_amount:
    users.loc[users['user_id'] == index, 'amount'] = value

CPU times: user 2min 16s, sys: 683 ms, total: 2min 17s
Wall time: 2min 17s


In [100]:
users.head()

,user_id,n_orders,last_ordered_date,amount
0,543272936,32.0,20191031.0,4388.45
1,551377651,13.0,20191025.0,4311.08
2,524325294,3.0,20191008.0,1752.46
3,555083442,2.0,20191005.0,754.24
4,515246296,8.0,20191024.0,3754.73


И так последний штрих в вычислениях. Нужно  преобразовать `last_ordered_date` в `recency`, то есть вычесть самый-самый последний день, указанные в данных **(20191031)**, из последнего совершения покупки определенного клиента.

In [101]:
max_date = smartphones['event_date'].max()
max_date

20191031

In [102]:
users['order_time_offset'] = max_date - users['last_ordered_date'])

In [103]:
users.head()

,user_id,n_orders,last_ordered_date,amount,order_time_offset
0,543272936,32.0,20191031.0,4388.45,-0.0
1,551377651,13.0,20191025.0,4311.08,6.0
2,524325294,3.0,20191008.0,1752.46,23.0
3,555083442,2.0,20191005.0,754.24,26.0
4,515246296,8.0,20191024.0,3754.73,7.0


In [104]:
users = users.drop('last_ordered_date', axis=1)

In [105]:
users.head()

,user_id,n_orders,amount,order_time_offset
0,543272936,32.0,4388.45,-0.0
1,551377651,13.0,4311.08,6.0
2,524325294,3.0,1752.46,23.0
3,555083442,2.0,754.24,26.0
4,515246296,8.0,3754.73,7.0


In [106]:
rfm = users.drop('user_id', axis=1)

In [107]:
rfm.head()

,n_orders,amount,order_time_offset
0,32.0,4388.45,-0.0
1,13.0,4311.08,6.0
2,3.0,1752.46,23.0
3,2.0,754.24,26.0
4,8.0,3754.73,7.0


In [108]:
rfm['order_time_offset'] = rfm['order_time_offset'].astype(int)
rfm['n_orders'] = rfm['n_orders'].astype(int)
rfm['amount'] = rfm['amount'].astype(int)

In [109]:
rfm.head()

,n_orders,amount,order_time_offset
0,32,4388,0
1,13,4311,6
2,3,1752,23
3,2,754,26
4,8,3754,7


### Переименование и упорядочевания колонок.
Для красоты и понятности переименую колонки и упорядочую как в аббревиатуре. 

In [110]:
rfm = rfm.rename( \
    columns={'n_orders': 'Recency', 'order_time_offset':'Frequency', 'amount': 'Monetary'}
)

In [111]:
rfm.head()

,Recency,Monetary,Frequency
0,32,4388,0
1,13,4311,6
2,3,1752,23
3,2,754,26
4,8,3754,7


In [112]:
rfm = rfm[['Recency', 'Frequency', 'Monetary']]

In [113]:
rfm.head()

,Recency,Frequency,Monetary
0,32,0,4388
1,13,6,4311
2,3,23,1752
3,2,26,754
4,8,7,3754


Визуализация величин. Распределение по гисто и QQPlot.

In [ ]:
fig, axes = plt.subplots(1, 3)
rec_hist = sns.distplot(ax=axes[0], a=rfm['Recency'], kde=True)
rec_hist.set_title('Distribution of `Recency`')

fre_hist = sns.distplot(ax=axes[1], a=rfm['Frequency'], kde=True)
fre_hist.set_title('Distribution of `Frequency`')

mon_hist = sns.distplot(ax=axes[2], a=rfm['Monetary'], kde=True)
mon_hist.set_title('Distribution of `Monetary`');

In [ ]:
from scipy import stats

fig, axes = plt.subplots(1, 3)
stats.probplot(x=rfm['Recency'], dist="norm", plot=axes[0])
stats.probplot(x=rfm['Frequency'], dist="norm", plot=axes[1])
stats.probplot(x=rfm['Monetary'], dist="norm", plot=axes[2]);

In [ ]:
fig, axes = plt.subplots(1, 3)
rec_box = sns.boxplot(ax=axes[0], y=rfm['Recency'])
rec_box.set_title('Boxplot of `Recency`')

fre_box = sns.boxplot(ax=axes[1], y=rfm['Frequency'])
fre_box.set_title('Boxplot of `Frequency`')

mon_box = sns.boxplot(ax=axes[2], y=rfm['Monetary'])
mon_box.set_title('Boxplot of `Monetary`');

### Скалирование величин.
Многие алгоритмы кластеризации под капотом вычисляют дистанции (Euclidean, Manhattan). Поэтому скалирование величин **обязательный** гость программы.

Из-за того, что в `Recency` и `Monetary` большой разброс значений, я буду использовать `MinMax` скалирование.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

for column in rfm.columns:
    rfm[column] = scaler.fit_transform(rfm[[column]])

In [ ]:
rfm.head()

In [ ]:
del scaler

In [ ]:
rfm_scaled = rfm.copy()

### Удаление выбросов.

In [ ]:
for column in rfm.columns:
    rfm_scaled = rfm_scaled[(rfm_scaled[column] < 3) & (rfm_scaled[column] > -3)]

In [ ]:
rfm_scaled.head()

In [ ]:
fig, axes = plt.subplots(1, 3)
rec_hist = sns.distplot(ax=axes[0], a=rfm_scaled['Recency'], kde=True)
rec_hist.set_title('Distribution of Scaled `Recency`')

fre_hist = sns.distplot(ax=axes[1], a=rfm_scaled['Frequency'], kde=True)
fre_hist.set_title('Distribution of Scaled `Frequency`')

mon_hist = sns.distplot(ax=axes[2], a=rfm_scaled['Monetary'], kde=True)
mon_hist.set_title('Distribution of Scaled `Monetary`');

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(rfm_scaled['Recency'], rfm_scaled['Monetary'], rfm_scaled['Frequency'])
ax.set_xlabel("Recency")
ax.set_ylabel("Monetary")
ax.set_zlabel("Frequency")
plt.show()

### Выбор и тренировка моделей.

Выбор моделей, я конечно же начну с самого простого и популярного алгортима K-means.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Найдем кол-во кластеров с помощью `Elbow curve`

results = []
range_clust = range(2, 15)

for num in range_clust:
    kmeans = KMeans(n_clusters=num)
    kmeans.fit(rfm_scaled)
    
    results.append({'N_clusters': num, 'Inertia': kmeans.inertia_})

In [ ]:
results = pd.DataFrame(results)

In [ ]:
sns.lineplot(data=results, x='N_clusters', y='Inertia');

***По методу Elbow выбираем ответ 4.*** <img src='https://miro.medium.com/max/1400/1*eVyOdx4gIcGWQ3lF4xAu6g.png' width='400' heigh='200'/>

In [ ]:
kmeans_elbow = KMeans(n_clusters=4)
kmeans_elbow.fit(rfm_scaled)

In [ ]:
rfm_scaled['Cluster_id'] = kmeans_elbow.labels_

In [ ]:
rfm_scaled['Cluster_id'] = rfm_scaled['Cluster_id'] + 1

In [ ]:
rfm_scaled.head()

### 

In [ ]:
fig, axes = plt.subplots(1,3)

rec_cluster = sns.boxplot(ax=axes[0], x='Cluster_id', y='Recency', data=rfm_scaled);
rec_cluster.set_title("Clustered `Recency` boxplot")
freq_cluster = sns.boxplot(ax=axes[1], x='Cluster_id', y='Frequency', data=rfm_scaled);
freq_cluster.set_title("Clustered `Frequency` boxplot")
mon_cluster = sns.boxplot(ax=axes[2], x='Cluster_id', y='Monetary', data=rfm_scaled);
mon_cluster.set_title("Clustered `Monetary` boxplot");

### Выводы

Недавно покупали товар пользователи из кластера 1, 2, 3 

Чаще всего покупают 3 кластер, 1 кластер и некоторые пользователи из 4 кластера

Юзеры из 4 кластер айди покупают больше всех, в остальных кластерах покупатели тратять примерно одиниково, не считая выбросов.

Может 4 кластер - это оптовики? Покупают много часто и последнюю покупку совершали давно. Может им предложить отдельную цену за товар. Тогда мы сможем их вернуть и продать им еще больше смартфонов.